In [1]:
import os
import numpy as np
import tifffile as tiff

In [3]:


def replace_with_3_sharpest_layers(image_dir):
    # Traverse all directories and subdirectories
    for root, _, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(('.tiff', '.tif')):  # Check for TIFF and TIF files
                img_path = os.path.join(root, file)
                
                # Read the image
                image = tiff.imread(img_path)
                
                # Handle 2D grayscale images
                if len(image.shape) == 2:
                    print(f"Image {img_path} is a 2D grayscale image (shape: {image.shape}).")
                    
                    # Convert to 3-channel by duplicating the grayscale image
                    three_channel_image = np.stack([image] * 3, axis=0)  # Creates (3, H, W)
                    
                    # Save the 3-channel image to replace the original
                    tiff.imwrite(img_path, three_channel_image.astype(image.dtype))
                    print(f"Replaced {img_path} with a 3-channel image.")
                    continue
                
                # Handle images with fewer than 3 layers
                if image.ndim > 2 and image.shape[0] < 3:
                    print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")
                    break  # Skip processing for this image

                # If the image has more than 3 layers, proceed
                if image.ndim > 2 and image.shape[0] > 3:
                    print(f"Processing {img_path} with {image.shape[0]} layers.")
                    
                    # Normalize the 16-bit image to [0, 1] for consistent sharpness calculation
                    image_normalized = image.astype(np.float32) / 65535.0
                    
                    # Calculate sharpness for each layer
                    sharpness_scores = []
                    for i in range(image.shape[0]):
                        layer = image_normalized[i]
                        gy, gx = np.gradient(layer)  # Compute gradients
                        gnorm = np.sqrt(gx**2 + gy**2)  # Gradient magnitude
                        sharpness = np.average(gnorm)  # Average sharpness
                        sharpness_scores.append((sharpness, i))  # Store with index
                    
                    # Sort layers by sharpness score and select the top 3 sharpest layers
                    sharpest_layers = sorted(sharpness_scores, reverse=True, key=lambda x: x[0])[:3]
                    top_3_indices = [index for _, index in sharpest_layers]
                    
                    # Create a new image with only the 3 sharpest layers
                    new_image = image[top_3_indices, :, :]
                    
                    # Save the new image to replace the original
                    tiff.imwrite(img_path, new_image.astype(np.uint16))
                    print(f"Replaced {img_path} with the 3 sharpest layers.")

# Example usage
image_dir = r"C:\Users\k54739\Today_data\segmentation\test_img"
replace_with_3_sharpest_layers(image_dir)


In [3]:
import numpy as np
import tifffile as tiff

# Read the image
image = tiff.imread(r"C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\Day10_drugscreened&singledose_untreated\ds_41_D07-T01.tiff")

# Normalize the 16-bit image to [0, 1] for consistent sharpness calculation
image_normalized = image.astype(np.float32) / 65535.0

# Calculate sharpness for each layer
sharpness_scores = []
for i in range(image.shape[0]):
    layer = image_normalized[i]
    gy, gx = np.gradient(layer)  # Compute gradients
    gnorm = np.sqrt(gx**2 + gy**2)  # Gradient magnitude
    sharpness = np.average(gnorm)  # Average sharpness
    sharpness_scores.append((sharpness, i))  # Store with index

# Find the layer with the highest sharpness score
sharpest_layer = max(sharpness_scores, key=lambda x: x[0])
sharpest_index = sharpest_layer[1]  # Index of the sharpest layer

# Extract the sharpest layer
sharpest_image = image[sharpest_index]

print(f"The sharpest layer is at index: {sharpest_index}")


The sharpest layer is at index: 3


In [19]:
import numpy as np
import tifffile as tiff

# Read the image
image = tiff.imread(r"C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\Day10_drugscreened&singledose_untreated\ds_41_D10-T01.tiff")

# Normalize the 16-bit image to [0, 1] for consistent sharpness calculation
image_normalized = image.astype(np.float32) / 65535.0

# Calculate sharpness for each layer
sharpness_scores = []
for i in range(image.shape[0]):
    layer = image_normalized[i]
    gy, gx = np.gradient(layer)  # Compute gradients
    gnorm = np.sqrt(gx**2 + gy**2)  # Gradient magnitude
    sharpness = np.average(gnorm)  # Average sharpness
    sharpness_scores.append((sharpness, i))  # Store with index

# Sort the sharpness scores in descending order
sharpness_scores_sorted = sorted(sharpness_scores, key=lambda x: x[0], reverse=True)

# Get the top 3 sharpest layers
top_3_sharpest = sharpness_scores_sorted[:3]

# Extract and print the indices of the top 3 sharpest layers
print("Top 3 sharpest layers:")
for rank, (sharpness, index) in enumerate(top_3_sharpest, start=1):
    print(f"Rank {rank}: Layer index {index} with sharpness score {sharpness:.6f}")

# Extract the top 3 sharpest layers
#top_3_layers = [image[index] for _, index in top_3_sharpest]

# Optionally, you can save the top 3 sharpest layers as a new TIFF file
#top_3_image = np.stack(top_3_layers, axis=0)
#tiff.imwrite(r'C:\path\to\save\top_3_sharpest_layers.tiff', top_3_image)

#print("Saved the top 3 sharpest layers as 'top_3_sharpest_layers.tiff'.")


Top 3 sharpest layers:
Rank 1: Layer index 0 with sharpness score 0.010400
Rank 2: Layer index 1 with sharpness score 0.010206
Rank 3: Layer index 2 with sharpness score 0.010028


In [14]:
import cv2
import numpy as np
import tifffile

# Function to align a target channel to the reference channel
def align_image(ref_image, target_image):
    # Convert images to 8-bit for better feature detection
    ref_image_8bit = cv2.normalize(ref_image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    target_image_8bit = cv2.normalize(target_image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Find keypoints and descriptors using ORB
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(ref_image_8bit, None)
    kp2, des2 = orb.detectAndCompute(target_image_8bit, None)

    # Match descriptors using brute-force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matching keypoints
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Compute the affine transformation matrix
    matrix, _ = cv2.estimateAffinePartial2D(dst_pts, src_pts)

    # Warp the target image to align with the reference image
    aligned_image = cv2.warpAffine(target_image, matrix, (ref_image.shape[1], ref_image.shape[0]))
    return aligned_image

# Load the multi-channel TIFF image using tifffile (this supports 5 channels)
image = tifffile.imread(r"C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\ds_41_D08-T01.tiff")

# Ensure the image has 5 channels (e.g., 5 layers)
if image.shape[0] == 5:
    # Split the image into its 5 channels (each channel will be a 2D array)
    channels = [image[i] for i in range(5)]
else:
    print("Error: The image does not have 5 channels.")
    exit()

# Reference channel (e.g., the first channel)
ref_channel = channels[0]

# Align all other channels to the reference channel
aligned_channels = [ref_channel]  # Start with the reference channel
for i in range(1, len(channels)):
    aligned_channel = align_image(ref_channel, channels[i])
    aligned_channels.append(aligned_channel)

# Stack the aligned channels into a single 5-channel image
aligned_image = np.stack(aligned_channels, axis=0)

# Save the aligned image as a TIFF file
tifffile.imwrite(r'C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\aligned_imaged.tiff', aligned_image)

print("Aligned image has been saved as 'aligned_image.tiff'.")


Aligned image has been saved as 'aligned_image.tiff'.


In [8]:
image = cv2.imread(r"C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\Day10_drugscreened&singledose_untreated\ds_41_D07-T01.tiff", cv2.IMREAD_UNCHANGED)

In [9]:
image.shape

(2054, 2456)

In [10]:
import tifffile as tiff

In [11]:
image = tiff.imread(r"C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\Day10_drugscreened&singledose_untreated\ds_41_D07-T01.tiff")

In [12]:
image.shape

(5, 2054, 2456)

# this didn't worl properly



import numpy as np
import tifffile
from skimage.metrics import structural_similarity as ssim

# Function to calculate alignment score (similarity) between two channels
def calculate_similarity(ref_channel, target_channel):
    # Normalize channels to 8-bit for consistency in metrics
    ref_channel_8bit = cv2.normalize(ref_channel, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    target_channel_8bit = cv2.normalize(target_channel, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    # Compute SSIM (Structural Similarity Index)
    score, _ = ssim(ref_channel_8bit, target_channel_8bit, full=True)
    return score

# Load the multi-channel TIFF image using tifffile
image = tifffile.imread(r"C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\ds_41_D08-T01.tiff")

# Ensure the image has 5 channels
if image.shape[0] != 5:
    print("Error: The image does not have 5 channels.")
    exit()

# Split the image into its 5 channels (each channel is a 2D array)
channels = [image[i] for i in range(5)]

# Reference channel (e.g., the first channel)
ref_channel = channels[0]

# Compute similarity scores between the reference channel and other channels
similarity_scores = []
for i, channel in enumerate(channels):
    score = calculate_similarity(ref_channel, channel)
    similarity_scores.append((i, score))

# Sort channels by similarity score in descending order
sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Select the indices of the top 3 channels based on similarity scores
top_indices = [index for index, _ in sorted_scores[:3]]
top_channels = [channels[i] for i in top_indices]

# Stack the selected channels into a single 3-channel image
selected_image = np.stack(top_channels, axis=0)

# Save the selected channels as a new TIFF file
tifffile.imwrite(
    r'C:\Users\k54739\Today_data\time_prediction\Day10today10Normal\selected_image.tiff', 
    selected_image, 
    photometric='minisblack'  # Ensures the multi-channel image is saved correctly
)

print(f"Saved the selected 3 channels as 'selected_image.tiff'. Selected channel indices: {top_indices}")
